In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
file_path = r"data.txt"
loader = TextLoader(file_path=file_path, encoding="utf-8")
docs = loader.load()

In [12]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(docs)
print(len(chunks))

13276


In [ ]:
persist_directory = r"C:\Users\Ansh Lulla\VS-Code\sam\chroma-db"
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=embedding_model, 
                                    persist_directory=persist_directory)
retriever = vectorstore.as_retriever()  
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F409F1C6B0>, search_kwargs={})

In [22]:
system_prompt = """
    As a banking-focused assistant, your responses are strictly limited to the following topics:
    - Card Management
    - Transaction Inquiries
    - Loans and Mortgages
    - Transfers and Payments
    - Customer Service
    - Fees and Charges
    - Identity Verification
    - Account Management
    - General Banking Support

    ### Instructions for Responses:

    1. **For Banking-Related Queries (Intent = "banking")**:
    - Provide a complete and concise answer directly based on the context and question.
    - Avoid reasoning, unnecessary details, or incomplete answers.
    - End with: "Is there anything else I can assist you with today?"

    2. **For Non-Banking Queries (Intent = "nonBanking")**:
    - **Reply only with**: "I'm here to assist with banking-related queries only. Could you please ask a question about topics like transactions, card management, or loans?"

    Intent: {intent}
    Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [23]:
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="Gemma2-9b-It", temperature=0.6, api_key=api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F41A087E60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F41A084BC0>, model_name='Gemma2-9b-It', temperature=0.6, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
qa_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
qa_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input', 'intent'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'intent'], input_types={}, partial_variables={}, template='\n    As a banking-focused assistant, your responses are strictly limited to the following topics:\n    - Card Management\n    - Transaction Inquiries\n    - Loans and Mortgages\n    - Transfers and Payments\n    - Customer Service\n    - Fees and Charges\n    - Identity Verification\n    - Account Management\n    - General Banking Support\n\n    ### Instructions for Responses:\n\n    1. **For Banking-Related Queries (Intent = "banking")**:\n    - Provide a complete and concise answer directly based on the context and question.\n    - Avoid reasoning, unneces

In [25]:
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=qa_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F409F1C6B0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input', 'intent'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'intent'], input_types={}, partial_variables={}, template='\n    As a banking-focused assistant, your responses are strictly limited to the following topics:\n    - Card Management

In [29]:
response = rag_chain.invoke({"input": "What is a loan?", "intent": "banking"})
print(response["answer"])

A loan is a sum of money borrowed from a lender with the agreement to repay it, usually with interest, over a specified period. 

Is there anything else I can assist you with today? 



In [ ]:
contextualize_q_system_prompt = ("""
    Given a chat history and the latest user query, which might be a reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, reformulate it if needed otherwise return it as it is.    
""")

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

contextualize_q_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [33]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F409F1C6B0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [34]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [35]:
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F409F1C6B0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.An

In [38]:
chat_history = []
question = "What are mutual funds and why should I invest in them?"
response1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "intent": "banking"})

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response1["answer"])
])

question2 = "Tell me more about it."
response2 = rag_chain.invoke({"input": question2, "chat_history": chat_history,"intent": "banking"})
print(response1["answer"])
print("---------------------")
print(response2["answer"])

Mutual funds are investment vehicles that pool money from multiple investors to invest in a diversified portfolio of assets such as stocks, bonds, and other securities. They are managed by professional fund managers who make investment decisions on behalf of the investors. 

Mutual funds offer several benefits, including diversification, professional management, liquidity, and potential for growth. 

Is there anything else I can assist you with today?




---------------------
Mutual funds offer a way to invest in a diversified portfolio of assets without having to individually research and select each security. 

Here's a breakdown of some key benefits:

* **Diversification:** By investing in a mutual fund, your money is spread across a variety of investments, which helps to reduce risk. 
* **Professional Management:**  Experienced fund managers make investment decisions based on market research and analysis, aiming to achieve the fund's investment objectives.
* **Liquidity:**  Many m

In [40]:
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response2["answer"])
])

question3 = "Which Indian banks offer are good for it?"
response3 = rag_chain.invoke({"input": question3, "chat_history": chat_history,"intent": "banking"})
print(response3["answer"])

Many Indian banks offer mutual fund services. Some popular options include:

* HDFC Bank
* ICICI Bank
* Axis Bank
* SBI
* Kotak Mahindra Bank

It's recommended to research and compare different banks based on their:

*  Fees
*  Fund selection
*  Investment platforms
*  Customer service 

Is there anything else I can assist you with today? 





